Created on March 21st 2020 by Patrick Rotzetter
Last update on Feb 28th 2021

protzetter@bluewin.ch
https://www.linkedin.com/in/rotzetter/

**Small experiment for order mail processing**




In [1]:
# Import libraries
import spacy
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher,PhraseMatcher
from spacy.symbols import nsubj, VERB, dobj, NOUN, root, xcomp
from spacy import displacy
from spacy.matcher import Matcher
from pathlib import Path
import random  

In [2]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.1) =================
ℹ spaCy installation:
/opt/anaconda3/envs/spacy30/lib/python3.8/site-packages/spacy

NAME              SPACY            VERSION                            
en_core_web_lg    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_sm    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_trf   >=3.0.0,<3.1.0   3.0.0   ✔



In [3]:
from platform import python_version
print(python_version())
!pip show spacy

3.8.5
Name: spacy
Version: 3.0.1
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /opt/anaconda3/envs/spacy30/lib/python3.8/site-packages
Requires: murmurhash, preshed, cymem, tqdm, catalogue, wasabi, srsly, jinja2, typer, pydantic, requests, blis, spacy-legacy, pathy, packaging, setuptools, thinc, numpy
Required-by: texthero, en-core-web-trf, en-core-web-sm, en-core-web-lg


In [4]:
# load spacy model latets modle based on pre-trained transformer model Roberta

from spacy.lang.en import English
import en_core_web_trf
nlp = en_core_web_trf.load()

In [5]:
#read order dialog file

text = open('ordermail.txt').read().replace('\n', ' ')

print(text)


Hello,  I would like to order a notebook with 16GB and 256 GB disk, I would like to spend less than 1000 Francs, what would be the options  Thanks a lot  Patrick


In [6]:
#process the mail trough standard spacy pipeline
doc=nlp(text)


In [7]:
# print text entities detected
for ent in doc.ents :
    print(ent.text, ent.label_,)


16GB QUANTITY
256 GB QUANTITY
1000 Francs QUANTITY
Patrick PERSON


In [8]:
#Let us visualize the result directly in the text
displacy.render(doc, style='ent', minify=True)

The default model does not seem to detect notebook and disk as entities, but identifies the sender as a person and identifies the RAM and disk size as quantities. This is a good start, but still far away from a practical solution. So, let us add some domain specific entities that will help us later on.

In [9]:
# add domain specific entities and add to the pipeline
patterns = [{"label": "CURRENCY", "pattern":  [{"lower": "francs"}]},
            {"label": "PART", "pattern":  [{"lower": "disk"}]}]


config = {
   "phrase_matcher_attr": None,
   "validate": True,
   "overwrite_ents": True,
   "ent_id_sep": "||",
}
ruler=nlp.add_pipe('entity_ruler',config=config)


In [10]:
ruler.add_patterns(patterns)

In [11]:
#process the mail again with added entities
doc=nlp(text)
for ents in doc.ents:
    # Print the entity text and its label
    print(ents.text, ents.label_,)



16GB QUANTITY
256 GB QUANTITY
disk PART
Francs CURRENCY
Patrick PERSON


In [12]:
#Let us visualize the result directly in the text
displacy.render(doc, style='ent', minify=True)

In [ ]:
# Let us try to identify specific phrases or sequence of words, for example to detect the memory size
matcher = PhraseMatcher(nlp.vocab)
terms = ["16 GB","256 GB"]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(t) for t in terms]
matcher.add("MEMORY", None, *patterns)

doc = nlp(text)
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

In [ ]:
# Part of speech tagging
# This is where the trained pipeline and its statistical models come in, 
# which enable spaCy to make predictions of which tag or label most likely applies in this context
for token in doc:
  print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

In [ ]:
# visualize the dependency graph
displacy.render(doc, style="dep", minify=True, jupyter=True)

Spacy provides all the required tagging to find the action verbs, we want to know if the customer wants to order something or is just interested by some information for example. Let us iterate through all tokens in the text and search for an open clausal complement ( refer to for all possible dependency tags https://spacy.io/api/annotation#pos-tagging )

In [ ]:
# Identify action verbs
verbs = set()
for possible_verbs in doc:
    if possible_verbs.dep == xcomp and possible_verbs.head.pos == VERB :
        verbs.add(possible_verbs)
print(verbs)

Let us find possible items in the text using the dependency tag ‘dobj’ for direct objects of a verb.

In [ ]:
# Let us find possible items in the text using the dependency tag ‘dobj’ for direct objects of a verb.
items = set()
for possible_subject in doc:
    if possible_subject.dep == dobj and possible_subject.head.pos == VERB:
        items.add(possible_subject)
print(items)

In [ ]:
# We will compare similarities between identified objects and the word ‘laptop’. 
# The word ‘notebook’ is much closer to ‘laptop’ than Francs.
import en_core_web_lg
nlp = en_core_web_lg.load()
orderobject=nlp("laptop")
for  sub in items:
  print(sub.text,nlp(sub.text).similarity(orderobject))


In [ ]:
# We will compare similarities between identified verbs and the verb ‘order’. 
# Then based on this we will identiofy the direct object of the verb
orderword=nlp("order") 
for  verb in verbs:
  if (nlp(verb.text).similarity(orderword)) >=0.8:
    for v in verb.children:
      if v.dep==dobj:
        print(v.text)


In [ ]:
# we can also identify specific slots using numeric modifiers for example to understand the desired quantities
# with corresponding modified objects
for token in doc:  
    if token.dep_ == 'nummod':    
      print(f"Numerical modifier: {token.text} --> object: {token.head}")